In [49]:
#seq2seq
import numpy as np
import tensorflow as tf


In [50]:
# #potrzebne są jakieś dane za cel stawiamy sobie tłumaczenie
with open('pol.txt', 'r', encoding='utf-8') as f:
    file = f.read()
    f.close()


# input_tekst =[] #to będzie lista z tekstami wejściowymi
# teksty_docelowe=[]#to będzie lista z tekstami docelowymi

    
# # linie=file.split('\n') #to takie troche dokumenty tak to możemy rozmieć
# # en,pl,_=linie[0][:10000].split('\t') # w efelcie mam trzy listy z tekstami
# # input_tekst.append(en)
# # teksty_docelowe.append("\t"+pl+"\n") #dodajemy znaczniki początku i końca tabulator to początek a nowa linia koniec
# #tekst docelowy powinien mieć poczatek i koniec znaczniki w tekście taka by wiedzieć kiedy się kończy



In [51]:
#testy i identyfiakcja błędu
# for idw,wiersz in enumerate(linie):
#     print(idw)
#     en, pl, _ =wiersz[:min(10000,len(linie)-1)].split('\t') 

In [52]:
input_texts = []
teksty_docelowe=[]


linie=file.split('\n')
linie=linie[:len(linie)-1] #ostatnia linia jest pusta


#łownik zlozony nie ze słów tylko liter znaków
chars=set()
znaki=set()


for wiersz in linie:
    en,pl,_=wiersz[:min(10000,len(linie)-1)].split('\t') 
    input_texts.append(en)
    teksty_docelowe.append("\t"+pl+"\n")
#chcemy działać na literach nie na całych słowach
    for char in en:
        chars.add(char)
    for znak in pl:
        znaki.add(znak)
        
znaki.add('\t')
znaki.add('\n')

In [53]:
#zwróćmy na uwage na wlikości słownika

no_chars = len(chars)
no_chars
ile_znkaow=len(znaki)
ile_znkaow
#powinniśmy to teraz w pewny sposób zakodować więc musimy nzkeźć najdłuższy znak

longest_seq= max([len(seq) for seq in input_texts])
najdluzsza_sek=max([len(seq) for seq in teksty_docelowe])

longest_seq,najdluzsza_sek

(262, 315)

In [54]:
#przyisanie znakom jakieś indeksy
char2ind = {}
for id ,c in enumerate(chars):
    char2ind[c]=id
    
    
znak2ind = {}
for id ,c in enumerate(znaki):
    znak2ind[c]=id

In [55]:
#liczba danych, długosć sekwecnji,liczba słow w złowiku mamy takie wymiary

X_en= np.zeros((20000,longest_seq,no_chars))
X_pl= np.zeros((20000,najdluzsza_sek,ile_znkaow))
Y= np.zeros((20000,najdluzsza_sek,ile_znkaow))

In [56]:
#uzupełniamy te tablice wartościami

for idd,(d_en,d_pl) in enumerate(zip(input_texts[:20000],teksty_docelowe[:20000])): #numer dokumentu, texty wiersze w językach jest tutaj niedokładność bo zip wykonuje pętle do krótszego ale nieważne to teraz
    #petla po znkaach
    for idc,c in enumerate(d_en):
        X_en[idd,idc,char2ind[c]]=1.0 #jaki to ejst dkument idd, jaki znak idc, jaki znak char2ind[c] jaki token
    X_en[idd,idc+1:,char2ind[" "]]=1.0  #cokolwiek tam jest to uzupełniamy zeramspacjami taki paddingi
    
    for idz,z in enumerate(d_pl):
        X_pl[idd,idz,char2ind[z]]=1.0 #jaki to ejst dkument idd, jaki znak idc, jaki znak char2ind[c] jaki token
        if idz>0:
            Y[idd,idz-1,char2ind[z]]=1.0 #jaki to ejst dkument idd, jaki znak idc, jaki znak char2ind[c] jaki token przsuniecie o 1 w tył
    X_pl[idd,idz+1:,char2ind[" "]]=1.0 
    Y[idd,idz :,char2ind[" "]]=1.0 #dodajemy paddingi
    


KeyError: '\t'

In [ ]:
#copilot fix
input_texts = []
teksty_docelowe = []

linie = file.split('\n')
linie = linie[:len(linie) - 1]  # ostatnia linia jest pusta

# słownik złożony nie ze słów tylko liter znaków
znaki = set()

for wiersz in linie:
    en, pl, _ = wiersz[:min(10000, len(linie) - 1)].split('\t')
    input_texts.append(en)
    teksty_docelowe.append("\t" + pl + "\n")
    # chcemy działać na literach nie na całych słowach
    for char in en:
        znaki.add(char)
    for znak in pl:
        znaki.add(znak)

znaki.add('\t')
znaki.add('\n')
znaki.add(' ')  # Dodaj spację do zbioru znaków

# Tworzenie słownika char2ind
char2ind = {char: idx for idx, char in enumerate(sorted(znaki))}

# Inicjalizacja tablic (przykładowe wymiary, dostosuj do swoich danych)
num_samples = 20000
max_len_en = max(len(text) for text in input_texts[:num_samples])
max_len_pl = max(len(text) for text in teksty_docelowe[:num_samples])
num_chars = len(char2ind)

X_en = np.zeros((num_samples, max_len_en, num_chars), dtype='float32')
X_pl = np.zeros((num_samples, max_len_pl, num_chars), dtype='float32')
Y = np.zeros((num_samples, max_len_pl, num_chars), dtype='float32')

# Uzupełnianie tablic wartościami
for idd, (d_en, d_pl) in enumerate(zip(input_texts[:num_samples], teksty_docelowe[:num_samples])):
    for idc, c in enumerate(d_en):
        if c in char2ind:
            X_en[idd, idc, char2ind[c]] = 1.0
    X_en[idd, idc+1:, char2ind[" "]] = 1.0  # Padding

    for idz, z in enumerate(d_pl):
        if z in char2ind:
            X_pl[idd, idz, char2ind[z]] = 1.0
        if idz > 0 and z in char2ind:
            Y[idd, idz-1, char2ind[z]] = 1.0
    X_pl[idd, idz+1:, char2ind[" "]] = 1.0  # Padding
    Y[idd, idz:, char2ind[" "]] = 1.0  # Padding

In [62]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras import Model

In [63]:
#na enkoder test w jezyku naglieskim
enc_i=Input(shape=(None,no_chars))
enc_h= LSTM(256,return_state=True)# ile chcemy wymiarów iterazcji, zeby zwracał też stany ukryte
enc_out,state_h,state_c = enc_h(enc_i) #state_h to stan ukryty, state_c to stan komórki



#dekoder jezyk polski
dec_i=Input(shape=(None,ile_znkaow))
dec_h= LSTM(256,return_sequences=True,return_state=True)
dec_out, _,_=dec_h(dec_i,initial_state=[state_h,state_c]) #dodajemy stan ukryty
#warstwa wyjściowa
dec_dense=Dense(ile_znkaow,activation='softmax')(dec_out)


model=Model([enc_i,dec_i],dec_out)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])


In [64]:
hostory=model.fit([X_en,X_pl],Y,epochs=10,validation_split=0.2, ) 

Epoch 1/10


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, None, 79), found shape=(32, 24, 105)

In [ ]:
#copilot fix
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_texts = []
teksty_docelowe = []

linie = file.split('\n')
linie = linie[:len(linie) - 1]  # ostatnia linia jest pusta

# słownik złożony nie ze słów tylko liter znaków
znaki = set()

for wiersz in linie:
    en, pl, _ = wiersz[:min(10000, len(linie) - 1)].split('\t')
    input_texts.append(en)
    teksty_docelowe.append("\t" + pl + "\n")
    # chcemy działać na literach nie na całych słowach
    for char in en:
        znaki.add(char)
    for znak in pl:
        znaki.add(znak)

znaki.add('\t')
znaki.add('\n')
znaki.add(' ')  # Dodaj spację do zbioru znaków

# Tworzenie słownika char2ind
char2ind = {char: idx for idx, char in enumerate(sorted(znaki))}

# Konwersja tekstów na sekwencje indeksów
input_sequences = [[char2ind[char] for char in text] for text in input_texts]
target_sequences = [[char2ind[char] for char in text] for text in teksty_docelowe]

# Padding sekwencji
max_len_en = max(len(seq) for seq in input_sequences)
max_len_pl = max(len(seq) for seq in target_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_len_en, padding='post', value=char2ind[' '])
target_sequences = pad_sequences(target_sequences, maxlen=max_len_pl, padding='post', value=char2ind[' '])

# Inicjalizacja tablic (przykładowe wymiary, dostosuj do swoich danych)
num_samples = len(input_sequences)
num_chars = len(char2ind)

X_en = np.zeros((num_samples, max_len_en, num_chars), dtype='float32')
X_pl = np.zeros((num_samples, max_len_pl, num_chars), dtype='float32')
Y = np.zeros((num_samples, max_len_pl, num_chars), dtype='float32')

# Uzupełnianie tablic wartościami
for idd, (d_en, d_pl) in enumerate(zip(input_sequences, target_sequences)):
    for idc, c in enumerate(d_en):
        X_en[idd, idc, c] = 1.0

    for idz, z in enumerate(d_pl):
        X_pl[idd, idz, z] = 1.0
        if idz > 0:
            Y[idd, idz-1, z] = 1.0

# Sprawdzenie kształtów danych
print(f'X_en shape: {X_en.shape}')
print(f'X_pl shape: {X_pl.shape}')
print(f'Y shape: {Y.shape}')

# Dopasowanie modelu
history = model.fit([X_en, X_pl], Y, epochs=10, validation_split=0.2)